In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

23/03/18 22:41:40 WARN Utils: Your hostname, choeyunseoui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.20.14.17 instead (on interface en0)
23/03/18 22:41:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/18 22:41:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 저수준의 RDD API 패턴과 고수준 DSL과 데이터 프레임 API를 사용한 예시 비교

In [24]:
# RDD를 이용한 예제
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
sc = spark.sparkContext
# (name, age) 형태의 튜플로 된 RDD 생성
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)])
# 집계와 평균을 위한 람다 표현식, map, reduceByKey transformation
ageRDD = (dataRDD
          .map(lambda x: (x[0], (x[1], 1)))
          .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
          .map(lambda x: (x[0], x[1][0] / x[1][1]))
          )
print(ageRDD.collect())
spark.stop()

[('Brooke', 22.5), ('Denny', 31.0), ('TD', 35.0), ('Jules', 30.0)]


In [19]:
# 고수준 DSL, 데이터프레임 API 사용
from pyspark.sql.functions import avg
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)], ['name', 'age'])
avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

spark.stop()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



### 스키마 정의 방법

In [25]:
# 프로그래밍 스타일
from pyspark.sql.types import *
schema = StructType([StructField("author", StructType(), False),
                     StructField("title", StringType(), False),
                     StructField("pages", IntegerType(), False)])

In [26]:
# DDL 사용
schema = "author STRING, title STRING, pages INT"

### 로우 객체 이해하기

In [28]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter", "LinkedIn"])
blog_row[1]

'Reynold'

In [34]:
# 데이터 프레임으로 만들어서 사용
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

rows = [Row("Matei Zaharia", "CA"),Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

spark.stop()

+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



### 샌프란시스코 데이터 예제

데이터 불러오기

In [49]:
# schema 정의
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
fire_df = spark.read.csv('sf-fire-calls.csv', header=True)
fire_df.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

트랜젝션과 필터

In [58]:
from pyspark.sql.functions import *

# CallType이 "Medical Incident"가 아닌 데이터 필터
few_fire_df = (fire_df
               .select("IncidentNumber", "AvailableDtTM", "CallType")
               .where(col("CallType") != "Medical Incident"))
few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTM         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [59]:
# CallType의 종류가 몇 가지인지 세어주는 코드
(fire_df.select("CallType")
 .where(col("CallType").isNotNull())
 .agg(countDistinct("CallType").alias("DistinctCallTypes"))
 .show()
 )

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [60]:
# 모든 행에서 null이 아닌 개별 CallType을 추출
(fire_df
 .select("CallType")
 .where(col("CallType").isNotNull())
 .distinct()
 .show(10, False)
 )

+-----------------------------+
|CallType                     |
+-----------------------------+
|Elevator / Escalator Rescue  |
|Marine Fire                  |
|Aircraft Emergency           |
|Administrative               |
|Alarms                       |
|Odor (Strange / Unknown)     |
|Citizen Assist / Service Call|
|HazMat                       |
|Watercraft in Distress       |
|Explosion                    |
+-----------------------------+
only showing top 10 rows



칼럼의 이름 변경 및 추가 삭제

In [65]:
new_fire_df = fire_df.withColumnRenamed("Delay", "changeDelay")
new_fire_df.select("changeDelay").show(5)

+-----------+
|changeDelay|
+-----------+
|       2.95|
|        4.7|
|  2.4333334|
|        1.5|
|  3.4833333|
+-----------+
only showing top 5 rows



In [66]:
spark.stop()